In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from repeater import retry
import json


In [ ]:
icon_service = IconService(HTTPProvider("https://sejong.net.solidwallet.io", 3))
NID = 83
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# NID = 1
NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

# checking transaction result
def check_tx_result(res):
    if res['status'] == 0:
        print("Tx Failed, Message: " + res['failure']['message'])
    else:
        print("Success")
        
        
GOVERNANCE_SCORE = "cx06eaddea58321a98bd0e1c9bd70f4680d4f3a73a"
DEX_SCORE = "cx113466143604608cbf58d84674d9360b29f08530"
LOANS_SCORE = "cx2148bdb92054b2306076f3579c0ab7fced91a9ca"


# MAINNET ADDRESS
# GOVERNANCE_SCORE = "cx44250a12074799e26fdeee75648ae47e2cc84219"
# DEX_SCORE = "cxa0af3165c08318e988cb30993b3048335b94af6c"
# LOANS_SCORE = "cx66d4d90f5f113eba575bf793570135f9b10cece1"



In [ ]:
with open("./keystore/staking_test.pwd", "r") as f:
    key_data = f.read()
deployer_wallet = KeyWallet.load("./keystore/staking_test.json", key_data)

# using private key to load test wallet to perform transactions in case of need
# tester_wallet = KeyWallet.load(bytes.fromhex(""))
# tester_wallet.get_address()

In [ ]:
def deploy_contract(wallet, _to_address, _name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(wallet) \
        .to(_to_address) \
        .nid(NID) \
        .params(params) \
        .version(3) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_name)) \
        .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 40000000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

    return icon_service.send_transaction(signed_transaction)


def send_tx(wallet, _to_address, method, params, value):
    transaction = CallTransactionBuilder() \
        .from_(wallet.get_address()) \
        .to(_to_address) \
        .value(value) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(method) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    return get_tx_result(tx_hash)


def call_tx(_to_address, method, params):
    call = CallBuilder() \
        .to(_to_address) \
        .method(method) \
        .params(params) \
        .build()

    return icon_service.call(call)

#swap function for DEX
def swap(from_token, to_token, amount,wallet):
    data2 = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': contract_addresses['dex'], '_value': amount, '_data': data2.encode()}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    return get_tx_result(tx_hash)

In [ ]:
# Getting all the SCORE address from GOVERNANCE SCORE
# The result shouldn't have router and feehandler in keys
contract_addresses = call_tx(GOVERNANCE_SCORE, 'getAddresses', {})
print(contract_addresses)

In [ ]:
# Taking loans and swapping tokens on dex 

print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 11 * 10 **18}, 50 * 10 **18)
check_tx_result(res)


res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18, deployer_wallet)
check_tx_result(res)
# print txHash incase of need
# print(res['txHash'])

# res = swap(contract_addresses['sicx'], contract_addresses['bnUSD'],15*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], usds_score,50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], data['sicx'],50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['sicx'], omm_score,10*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(omm_score, data['sicx'],5*10**18, deployer_wallet)
# check_tx_result(res)


# PHASE 1

In Phase 1 , 
We deploy:
    Router contract,
    Fee handler contract and
    Governance Contract (update)

## DEPLOY ROUTER CONTRACT

In [ ]:
# We need to set all the staking sicx and dex address on router on_install 
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/router',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
router_score = res.get('scoreAddress', '')
check_tx_result(res)

## UPDATE GOVERNANCE CONTRACT

In [ ]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), GOVERNANCE_SCORE, './core_contracts/governance', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

# In this readonly-call , the feehandler and router score should show None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))


## DEPLOY FEE HANDLER

In [ ]:
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/fees',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
FEEHANDLER_SCORE = res.get('scoreAddress', '')
check_tx_result(res)

# PHASE 2 (Once the contract is accepted)

We set the fee handler and router addresses in governance contract and also sets all the function of fee handler.

#### SET ROUTER ADDRESS AND FEEHANDLER ADDRESS IN GOVERNANCE

In [ ]:
contract_addresses = {
 'loans': 'cx66d4d90f5f113eba575bf793570135f9b10cece1',
 'dex': 'cxa0af3165c08318e988cb30993b3048335b94af6c',
 'staking': 'cx43e2eec79eb76293c298f2b17aec06097be606e0',
 'rewards': 'cx10d59e8103ab44635190bd4139dbfd682fa2d07e',
 'reserve': 'cxf58b9a1898998a31be7f1d99276204a3333ac9b3',
 'dividends': 'cx203d9cd2a669be67177e997b8948ce2c35caffae',
 'daofund': 'cx835b300dcfe01f0bdb794e134a0c5628384f4367',
 'oracle': 'cxe647e0af68a4661566f5e9861ad4ac854de808a2',
 'sicx': 'cx2609b924e33ef00b648a409245c7ea394c467824',
 'bnUSD': 'cx88fd7df7ddff82f7cc735c871dc519838cb235bb',
 'baln': 'cxf61cd5a45dc9f91c15aa65831a30a90d59a09619',
 'bwt': 'cxcfe9d1f83fa871e903008471cca786662437e58d',
 'router': 'cx21e94c08c03daee80c25d8ee3ea22a20786ec231',
 'feehandler': 'cx5faae53c4dbd1fbe4a2eb4aab6565030f10da5c6'}
res = send_tx(btest_wallet, GOVERNANCE_SCORE, "setAddresses", {"_addresses": contract_addresses}, 0)
check_tx_result(res)

# the read-only fn should return the fee handler and router address instead of None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))

#### SET ACCEPTED DIVIDENDS TOKEN

In [ ]:
# call the tx through Governance SCORE.

res2 = send_tx(btest_wallet, GOVERNANCE_SCORE, "setAcceptedDividendTokens", {'_tokens': [contract_addresses['sicx'],
            contract_addresses['bnUSD'],
            contract_addresses["baln"]]}, 0)
check_tx_result(res2)
call_tx(contract_addresses['feehandler'], 'getAcceptedDividendTokens', {})

#### SET FEE PROCESSING INTERVAL

In [ ]:
res = send_tx(btest_wallet, GOVERNANCE_SCORE, "setFeeProcessingInterval", {"_interval": 900}, 0)
check_tx_result(res)
call_tx(contract_addresses['feehandler'], 'getFeeProcessingInterval', {})

#### SET ROUTES

In [ ]:
toToken = contract_addresses['baln']     
#sejong_address
# omm_score = "cx683c78174c740c3c6215de0c3ead1fad024324d6"
# usds_score ="cx2e008873de78f5bf82008a46b529d94924d9b6ae" 
# iusdc_score = "cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183"

# mainnet address
tokens = {
    'usds': "cxbb2871f468a3008f80b08fdde5b8b951583acf06",
    'iusdc': "cxae3034235540b924dfcc1b45836c293dcc82bfb7",
    'omm': "cx1a29259a59f463a67bb2ef84398b30ca56b5830a",
    'craft': "cx2e6d0fc0eca04965d06038c8406093337f085fcf",
    'metanyx': "cx369a5f4ce4f4648dfc96ba0c8229be0693b4eca2",
    'iusdt': "cx3a36ea1f6b9aa3d2dd9cb68e8987bcc3aabaaa88",
    'sicx': "cx2609b924e33ef00b648a409245c7ea394c467824",
    'bnUSD': "cx88fd7df7ddff82f7cc735c871dc519838cb235bb",
    'baln': "cxf61cd5a45dc9f91c15aa65831a30a90d59a09619"
}

routes = [
    ['omm', 'sicx', 'baln'],
    ['usds', 'bnUSD', 'baln'],
    ['iusdc', 'bnUSD', 'baln'],
    ['craft', 'sicx', 'baln'],
    ['metanyx', 'sicx', 'baln'],
    ['iusdt', 'bnUSD', 'baln'],
]

for route in routes:
    fromToken = tokens[route[0]]
    path = [tokens[token] for token in route[1:]]
    res = send_tx(btest_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken': fromToken, '_toToken': toToken, '_path': path}, 0)
    check_tx_result(res)
    print(call_tx(contract_addresses['feehandler'], 'getRoute', {"_fromToken": fromToken,"_toToken": contract_addresses['baln']}))


## UPDATE DEX CONTRACT

In [ ]:
# We need to set fee handler address on on_update.
tx = get_tx_result(
    deploy_contract(btest_wallet.get_address(), DEX_SCORE, './core_contracts/dex', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

## UPDATE LOANS CONTRACT

In [ ]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), LOANS_SCORE, './core_contracts/loans', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)


### Enable Fee Handler

In [ ]:
res = send_tx(btest_wallet, GOVERNANCE_SCORE, "enable_fee_handler",  {}, 0)
check_tx_result(res)

In [ ]:
print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 100 * 10 **18}, 500 * 10 **18)
check_tx_result(res)

#Swapping bnUSD to sICX
res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18,deployer_wallet)
check_tx_result(res)